In [1]:
import requests

def get_pdb_ids(uniprot_id):
    url = f"https://search.rcsb.org/rcsbsearch/v2/query?json={{\"query\":{{\"type\":\"terminal\",\"service\":\"text\",\"parameters\":{{\"attribute\":\"rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession\",\"operator\":\"exact_match\",\"value\":\"{uniprot_id}\"}}}},\"return_type\":\"polymer_entity\"}}"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    pdb_ids = {entry['identifier'].split('_')[0] for entry in data['result_set']}
    return pdb_ids

uniprot_id = 'Q06187'
pdb_ids = get_pdb_ids(uniprot_id)
print(f"PDB IDs for UniProt ID {uniprot_id}: {pdb_ids}")


PDB IDs for UniProt ID Q06187: {'1BTK', '1AWW', '1B55', '1QLY', '2GE9', '3GEN', '2Z0P', '1AWX', '1BWN', '1K2P'}


In [2]:
def download_pdb(pdb_id, output_file):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    response.raise_for_status()
    with open(output_file, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {pdb_id}.pdb to {output_file}")

# premier PDB ID de la list pour telecharger le fichier
for pdb_id in pdb_ids:
    output_file = f"{pdb_id}.pdb"
    download_pdb(pdb_id, output_file)
    break 


Downloaded 1BTK.pdb to 1BTK.pdb


In [3]:
import requests

# Fonction pour obtenir les scores de conservation de ConSurf pour une structure protéique spécifique
def get_conservation_scores(pdb_id):
    # URL de base de l'API de ConSurf
    base_url = "https://consurfdb.tau.ac.il/consurf_db/api/"
    
    # Endpoint pour obtenir les scores de conservation
    endpoint = "getConsurfData"
    
    # Paramètres de la requête
    params = {
        "pdb_id": pdb_id,
        "ChainID": "",  # Laisser vide si non pertinent
        "res_num": "",  # Laisser vide pour toute la protéine
        "FileFormat": "json"  # Format de sortie (json, text, excel)
    }
    
    # Faire la requête GET à l'API de ConSurf
    response = requests.get(f"{base_url}{endpoint}", params=params)
    
    # Vérifier si la requête a réussi
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erreur lors de l'obtention des scores de conservation pour {pdb_id}")
        return None

# Exemple d'utilisation : Obtenir les scores de conservation pour la structure 1BTK
pdb_id = "1BTK"
conservation_scores = get_conservation_scores(pdb_id)

# Imprimer les scores de conservation (ou les gérer selon le besoin)
if conservation_scores:
    print("Scores de conservation pour la structure", pdb_id)
    print(conservation_scores)


SSLError: HTTPSConnectionPool(host='consurfdb.tau.ac.il', port=443): Max retries exceeded with url: /consurf_db/api/getConsurfData?pdb_id=1BTK&ChainID=&res_num=&FileFormat=json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1131)')))